<a href="https://colab.research.google.com/github/melrahmtz/purple-box/blob/main/hands-on-practice/2702_hybrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hybrid Search** merges keyword search (full text search) and semantic search (similarity search). Since, our documents would mostly be about technical manual or product knowledge, I think Hybrid Search would have great performance.

The process of Hybrid Search:
1. **Keyword search**, which involves searching by specific words or phrases present in the content.
2. **Semantic search**, which involves understanding the context or meaning behind the search query rather than the specific words used.
3. **Fusion** takes the results from both search methods and merges them together based on a certain ranking or scoring system. We'll use Reciprocal Ranked Fusion (RRF) for now.

Source: https://supabase.com/docs/guides/ai/hybrid-search

In [1]:
#!pip install supabase sentence-transformers numpy -q
!pip install langchain --quiet
!pip install supabase --quiet
!pip install tiktoken --quiet
!pip install unstructured --quiet
!pip install numpy --quiet
!pip install transformers --quiet
!pip install -U langchain-community  --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.9 MB/s eta 0:00:00


In [2]:
import getpass
import os

if "HUGGINGFACEHUB_API_KEY" not in os.environ:
    os.environ["HUGGINGFACEHUB_API_KEY"] = getpass.getpass("HuggingFace API Key:")

if "SUPABASE_URL" not in os.environ:
    os.environ["SUPABASE_URL"] = getpass.getpass("Supabase URL:")

if "SUPABASE_SERVICE_KEY" not in os.environ:
    os.environ["SUPABASE_SERVICE_KEY"] = getpass.getpass("Supabase Service Key:")

HuggingFace API Key:··········
Supabase URL:··········
Supabase Service Key:··········


In [3]:
import os
import numpy
from supabase.client import Client, create_client
from langchain.vectorstores import SupabaseVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings

huggingfacehub_key = os.environ.get("HUGGINGFACEHUB_API_KEY")
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

# Use the bge-m3 embedding model (1024 dimensions)
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


<ipython-input-3-86af72cd4e71>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def generate_embedding(text):
    """Generates a 1024-dimensional embedding for the input text."""
    embedding = embeddings.embed_query(text)
    return embedding

def load_json_documents(json_path):
    """Loads a JSON file, extracts text, and converts to LangChain Document objects."""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    docs = []
    file_name = data.get("file_name", "unknown")

    for node in data.get("nodes", []):
        content = node.get("text", "").strip()
        if content:  # Ignore empty texts
            metadata = {
                "file_name": file_name,
                "index": node.get("index", "unknown")
            }
            docs.append(Document(page_content=content, metadata=metadata))

    return docs

def store_documents_in_supabase(json_path):
    """Loads, splits, embeds, and stores documents in Supabase."""
    documents = load_json_documents(json_path)

    # Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=300)
    chunks = text_splitter.split_documents(documents)
    print(f"Loaded {len(chunks)} chunks from {json_path}")

    # Store in Supabase
    for chunk in chunks:
        embedding = generate_embedding(chunk.page_content)
        data = {
            "content": chunk.page_content,
            "embedding": embedding
        }
        supabase.table("documents_hybrid_search").insert(data).execute()

    print("Documents successfully stored in Supabase.")

# Example: Load and store documents
store_documents_in_supabase("PDF1_all_nodes_with_images.json")


Loaded 80 chunks from PDF1_all_nodes_with_images.json
Documents successfully stored in Supabase.


In [8]:
def hybrid_search(query, match_count=10):
    """Calls the hybrid_search function in Supabase with bge-m3 embeddings."""
    query_embedding = generate_embedding(query)

    # Call Supabase RPC function
    response = supabase.rpc("hybrid_search", {
        "query_text": query,
        "query_embedding": query_embedding,
        "match_count": match_count
    }).execute()

    return response.data

# Example query
query = "What is sentiment analysis?"
results = hybrid_search(query, match_count=10)

# Display results
for doc in results:
    print(doc)


{'id': 164, 'fts_score': 0.0163934426229508, 'semantic_score': 0.0163934426229508, 'final_score': 0.0327868852459016, 'content': '## 2 Literature Review\n\nSentiment analysis, a subfield of Natural Language Processing (NLP), focuses on quantifying the emotional tone and intent conveyed in textual data. NLP itself is a rapidly growing area of machine learning (ML) that enables computers to process and understand human language through algorithms and statistical models such as the founding work by Jurafsky and Martin (2000) [14]. Within NLP, sentiment analysis applies techniques to assess positive, negative, or neutral sentiment in data, often used for applications like market forecasting and consumer behavior analysis [15].\n\nIn this context, sentiment is defined as the emotional polarity associated with a piece of text, often derived from news articles, tweets, or other media. It is typically measured using computational methods such as dictionary-based approaches or ML models. For ex

In [10]:
import pandas as pd

def hybrid_search_display_table(query, match_count=10):
    """Calls the hybrid_search function in Supabase and returns results as a DataFrame."""
    query_embedding = generate_embedding(query)

    # Call Supabase RPC function
    response = supabase.rpc("hybrid_search", {
        "query_text": query,
        "query_embedding": query_embedding,
        "match_count": match_count
    }).execute()

    # Convert results to DataFrame
    if response.data:
        df = pd.DataFrame(response.data, columns=["id", "fts_score", "semantic_score", "final_score", "content"])
        return df
    else:
        print("No results found.")
        return None

# Example Query
query = "What is sentiment analysis?"
df_results = hybrid_search_display_table(query, match_count=10)

# Display Table
if df_results is not None:
    from IPython.display import display
    display(df_results)


,id,fts_score,semantic_score,final_score,content
0,164,0.016393,0.016393,0.032787,"## 2 Literature Review\n\nSentiment analysis, ..."
1,162,0.016129,0.015625,0.031754,"Keywords: Hype-adjusted Probability Measure, N..."
2,229,0.015873,0.015873,0.031746,| [6] | J Deveikyte et al. 'A sentiment anal...
3,167,0.014706,0.016129,0.030835,The sentiment score has been defined by many a...
4,180,0.014286,0.015152,0.029437,<!-- formula-not-decoded -->\n\n## Notations\n...
5,182,0.015625,0.012987,0.028612,The new bias arises from the gap between the T...
6,192,0.015152,0.012821,0.027972,## 3.2.3 The 3 Criteria\n\nWe examine the impa...
7,224,0.015385,0.000000,0.015385,"## 6 Conclusion\n\nIn this paper, we present a..."
8,166,0.000000,0.015385,0.015385,and stock market behavior.\n\nWe adopted the V...
9,176,0.000000,0.014925,0.014925,"For the third data input, the 'overall daily a..."
